<a href="https://colab.research.google.com/github/malekzitouni/Generative-AI-with-diffusion-Models/blob/main/Classifier_Free_Diffusion_Guidance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

So far, we have been able to train a model to generate images of clothing using the [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist) dataset. However, there is no way for the user to specify what kind of images should be generated. Let's fix that by using [Classifier-Free Diffusion Guidance](https://arxiv.org/pdf/2207.12598.pdf), a relatively simple way to create a [Conditional Diffusion Model](https://github.com/TeaPearce/Conditional_Diffusion_MNIST/tree/main).


In [ ]:
import glob
import torch
import torch.nn.functional as F
from torch.optim import Adam
import torchvision.transforms as transforms

# Visualization tools
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.utils import save_image, make_grid

# User defined libraries
from utils import other_utils
from utils import ddpm_utils
from utils import UNet_utils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
IMG_SIZE = 16
IMG_CH = 1
BATCH_SIZE = 128
N_CLASSES = 10
data, dataloader = other_utils.load_transformed_fashionMNIST(IMG_SIZE, BATCH_SIZE)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

100%|██████████| 26.4M/26.4M [00:04<00:00, 6.16MB/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 131kB/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 2.48MB/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 4.69MB/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [ ]:
nrows = 10
ncols = 15

T = nrows * ncols
B_start = 0.0001
B_end = 0.02
B = torch.linspace(B_start, B_end, T).to(device)
ddpm = ddpm_utils.DDPM(B, device)

This time, our `UNet` is slightly different. We've added a few changes and moved the U-Net architecture into its own [UNet_utils.py](utils/UNet_utils.py) folder.

In the `__init__` function, we've added a new parameter: `c_embed_dim`. Like for timestep `t`, we can create an embedding for our class categories.

```python
        self.sinusoidaltime = SinusoidalPositionEmbedBlock(t_embed_dim)
        self.t_emb1 = EmbedBlock(t_embed_dim, up_chs[0])
        self.t_emb2 = EmbedBlock(t_embed_dim, up_chs[1])
        self.c_embed1 = EmbedBlock(c_embed_dim, up_chs[0])  # New
        self.c_embed2 = EmbedBlock(c_embed_dim, up_chs[1])  # New
```

Next, in the `forward` function, we have two new parameters: `c` and `c_mask`.
* `c` is a vector representing our categorical input. It can be a [one-hot encoding](https://www.kaggle.com/code/dansbecker/using-categorical-data-with-one-hot-encoding) or an embedding vector.
* `c_mask` is used to randomly set values within `c` to zero. This helps the model learn what an average output might be without a categorical input, like in the previous notebook.

```python
        c = c * c_mask
        c_emb1 = self.c_embed1(c)
        c_emb2 = self.c_embed2(c)
```

There are many different ways we can combine this embedded categorical information into the model. One popular method is with [scaling and shifting](https://arxiv.org/pdf/2210.08823.pdf). We can scale (multiply) our categorical embedding to the latent image and then (add) our time embedding `t_emb`. The scale and shift act as a sort of variance and average respectively.

```python
        up0 = self.up0(latent_vec)
        up1 = self.up1(c_emb1 * up0 + t_emb1, down2)  # Changed
        up2 = self.up2(c_emb2 * up1 + t_emb2, down1)  # Changed
        return self.out(torch.cat((up2, down0), 1))
```

We will convert our label to a format that can be processed by the model using the `get_context_mask` function below. Since our label is a single integer, we can use [F.one_hot](https://pytorch.org/docs/stable/generated/torch.nn.functional.one_hot.html) to turn it into an encoding vector.

To randomly set values from this one hot encoding to zero, we can use the [Bernoulli](https://mathworld.wolfram.com/BernoulliDistribution.html) distribution. This distribution is like flipping a weighted coin. "Heads" will land $p$ percent of the time and "Tails" will land $1-p$ percent of the time. In this case, our `drop_prob` represents "Tails".

<center><img src="/content/images/bernoulli.png"/></center>

In [ ]:
def get_context_mask(c, drop_prob):
    c_hot = F.one_hot(c.to(torch.int64), num_classes=N_CLASSES).to(device)
    c_mask = torch.bernoulli(torch.ones_like(c_hot).float() - drop_prob).to(device)
    return c_hot, c_mask

In [ ]:
model = UNet_utils.UNet(
    T, IMG_CH, IMG_SIZE, down_chs=(64, 64, 128), t_embed_dim=8, c_embed_dim=N_CLASSES
)
print("Num params: ", sum(p.numel() for p in model.parameters()))
model = torch.compile(model.to(device))

Num params:  2002561


In [ ]:
class_names = [
    "Top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

In [ ]:
optimizer = Adam(model.parameters(), lr=0.001)
epochs = 3
preview_c = 0

model.train()
for epoch in range(epochs):
    for step, batch in enumerate(dataloader):
        c_drop_prob = 0.1
        optimizer.zero_grad()

        t = torch.randint(0, T, (BATCH_SIZE,), device=device).float()
        x = batch[0].to(device)
        c_hot, c_mask = get_context_mask(batch[1], c_drop_prob)  # New
        loss = ddpm.get_loss(model, x, t, c_hot, c_mask)
        loss.backward()
        optimizer.step()

        if epoch % 1 == 0 and step % 100 == 0:
            class_name = class_names[preview_c]
            print(f"Epoch {epoch} | Step {step:03d} | Loss: {loss.item()} | C: {class_name}")
            c_drop_prob = 0 # Do not drop context for preview
            c_hot, c_mask = get_context_mask(torch.Tensor([preview_c]), c_drop_prob)
            ddpm.sample_images(model, IMG_CH, IMG_SIZE, ncols, c_hot, c_mask)
            preview_c = (preview_c + 1) % N_CLASSES